In [2]:
from bs4 import BeautifulSoup as bp
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psycopg2
import csv
import time



In [3]:
import getpass
import os

postgres_url = getpass.getpass("Enter your postgresql url: ")

In [20]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'vi-vn',
}

In [21]:
options=Options()
options.add_argument(f"user-agent={USER_AGENT}")
driver=webdriver.Chrome(options=options)

In [23]:
def get_hotel_id(soup):
    h_name=soup.find('h1',class_='css-4rbku5 css-901oao css-cens5h r-cwxd7f r-a5wbuh r-1x35g6 r-b88u0q r-fdjqy7')
    if h_name:
        h_name=h_name.text.strip()
    else:
        h_name=soup.find('h1',class_='css-4rbku5 css-901oao css-cens5h r-a5wbuh r-b88u0q r-fdjqy7')
        if h_name:
            h_name=h_name.text.strip()
        else:
            h_name=soup.find('h1',class_='css-4rbku5 css-901oao css-cens5h r-a5wbuh r-1x35g6 r-b88u0q r-fdjqy7')
            h_name=h_name.text.strip()
    query="""
        SELECT hotel_id
        FROM hotel
        WHERE name ILIKE %s;"""
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    cur.execute(query,(h_name,))
    hotel_id=cur.fetchone()
    cur.close()
    return hotel_id

In [8]:
def insert_db_hotel_price(hotel_id,name,capacity,price):
    print(f"insert rooms into db")
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    query="""INSERT INTO hotelprice (hotel_id, room_type, capacity, price)
    VALUES (%s, %s, %s, %s);"""
    hotel_id = hotel_id
    room_type = name
    capacity = capacity
    price = price
    cur.execute(query, (hotel_id, room_type, capacity, price))
    conn.commit()
    cur.close()
    conn.close()

In [15]:
def get_room_price(soup):
    #prices={}
    #price= soup.find('div',class_='css-901oao r-a5wbuh r-b88u0q r-135wba7 r-1ff274t')
    #prices['base_price']=price
    list=soup.find('div',attrs={'data-testid': 'room-list-tray','class':'css-1dbjc4n r-1oszu61 r-eqz5dr'})
    if list:
        rooms=list.findAll('div',class_='css-1dbjc4n r-14lw9ot r-1dzdj1l r-xy67v1 r-6gpygo r-1l7z4oj r-ymttw5')
        room_data={}
        for room in rooms:
            name=room.find('h3',class_='css-4rbku5 css-901oao css-bfa6kz r-cwxd7f r-a5wbuh r-b88u0q r-135wba7 r-fdjqy7',attrs={'role':'heading'})
            name=name.text.strip() if name else 'N/a'
        #room_data['name']
            price_list = []
            price_elements = room.find('div', class_='css-901oao r-1loqt21 r-a5wbuh r-ubezar r-b88u0q r-135wba7 r-1ff274t')
            for price_element in price_elements:
            #take string
                price_text = price_element.get_text().strip()
            #convert to INT
                price_int=int(price_text.replace('VND','').replace('.',''))
            #price_list.append(price_int)
            
        #room_data['price'] = price_list

            hotel_id=get_hotel_id(soup)
            count=0
            img_box=room.find('div',class_='css-1dbjc4n r-9cy4zw r-h1746q r-1mdj7ya r-15ik8dz r-youum4 r-1l7z4oj r-1e081e0 r-fd4yh7 r-136ojw6')
            dump=img_box.find('svg',attrs={'data-id':'IcUserAccountFill'})
            capa= dump.find_next_sibling('div',class_='css-901oao r-1ud240a r-a5wbuh r-1enofrn r-majxgm r-1cwl3u0 r-fdjqy7')
            if capa:
                div_capa=capa.get_text().strip().replace('x','')
            #room_data['capacity']=div_capa
                insert_db_hotel_price(hotel_id,name,div_capa,price_int)
            else:
                people=img_box.findAll('svg',attrs={'data-id':'IcUserAccountFill'})
                for svg in people:
                    count +=1
            #room_data['capacity']=count
                insert_db_hotel_price(hotel_id,name,count,price_int)
        return room_data
    else:
        hotel_id=get_hotel_id(soup)
        name='room for 2 people'
        count=int(2)
        price_int=1
        insert_db_hotel_price(hotel_id,name,count,price_int)


In [31]:
def get_hotel_amenities(soup):
    list=[]
    window=soup.find('div',attrs={'class':'css-1dbjc4n r-14lw9ot r-kdyh1x r-1fdih9r r-1udh08x r-ymttw5 r-1h4fu65', 'id':'section-facility'})
    amenities=window.findAll('div',class_='css-901oao r-cwxd7f r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
    if amenities:
        for amenity in amenities:
            stuff=amenity.text.strip()
            list.append(stuff)
    else:
        amenities=window.findAll('div',class_='css-901oao r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
        for amenity in amenities:
            stuff=amenity.text.strip()
            list.append(stuff)
    return list

In [11]:
def insert_db_hotel_amenities(list,a):
    print(f"insert amenities into db")
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    query="""
    UPDATE Hotel  SET amenities = %s
    WHERE hotel_id = %s;"""
    amenities=list
    cur.execute(query, (amenities,a))
    conn.commit()
    cur.close()
    conn.close()

In [32]:

for j in range(19,21):
    print(f"rooms in url {j}: ")
    with open(f"E:/GitHub/random_code/hotel_urls/hotel_urls_{j}.csv",newline='',encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        i=1
        data=[]
        for row in reader:
            print(f"rooms for {i}")
            url=row[0]
            #file_name = "room_price.txt"
            driver.get(url)
            time.sleep(10)
            soup=bp(driver.page_source,'lxml')
            get_room_price(soup)
            b=get_hotel_id(soup)
            list=soup.find('div',attrs={'data-testid': 'room-list-tray','class':'css-1dbjc4n r-1oszu61 r-eqz5dr'})
            if not list:
                data.append(b)
            a=get_hotel_amenities(soup)
            print(b,a)
            insert_db_hotel_amenities(a,b)
            i+=1
        with open(f'E:/GitHub/random_code/mis_Info/price_{j}.txt', mode='w', encoding='utf-8') as file:
            for id in data:
                file.write(str(id) + '\n')
            print(f"Đã ghi {len(data)} id vào file {j}")
            
            
           # with open(file_name, 'w', encoding='utf-8') as file:
             #   file.write(str(data))

rooms in url 19: 
rooms for 1
insert rooms into db
insert rooms into db
insert rooms into db
(367,) ['Truyền hình cáp', 'Bàn làm việc', 'Minibar', 'Phòng tắm vòi sen', 'TV', 'Đưa đón sân bay', 'Dịch vụ cho thuê xe đạp', 'Cho thuê xe hơi', 'Bãi đậu xe an ninh', 'Thức uống chào mừng miễn phí', 'Lễ tân 24h', 'Dịch vụ lưu trữ/bảo quản hành lý', 'Dịch vụ hỗ trợ đặt Tour', 'Bãi đậu xe', 'Thang máy', 'WiFi tại khu vực chung', 'Phòng giữ đồ', 'Phòng không hút thuốc', 'Đưa đón sân bay (thu phí)', 'Đưa đón sân bay', 'Cửa hàng thực phẩm', 'Cửa hàng', 'Wifi (miễn phí)']
insert amenities into db
rooms for 2
insert rooms into db
insert rooms into db
insert rooms into db
insert rooms into db
insert rooms into db
(368,) ['Bàn làm việc', 'Máy sấy tóc', 'Phòng tắm vòi sen', 'TV', 'Máy lạnh', 'Phòng không hút thuốc', 'Không khói thuốc', 'Cửa hàng quà tặng', 'Cửa hàng thực phẩm', 'Cửa hàng', 'Đồ ăn nhẹ', 'Wifi (miễn phí)', 'WiFi tại khu vực chung', 'Đưa đón sân bay (thu phí)', 'Quầy lễ tân']
insert amenit